# Introduction

# 1. Imports and Dependencies

In [ ]:
!pip install tensorflow opencv-python mediapipe scikit-learn matplotlib

In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import mediapipe as mp

2024-09-07 13:39:57.970057: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-07 13:39:59.089795: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-07 13:39:59.620017: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-07 13:39:59.948677: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-07 13:40:00.032469: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-07 13:40:06.206510: W tensorflow/compiler/tf2tensorrt/utils/p

# 2. Keypoints unsing MediaPipe Holistic

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [10]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()

    cv2.imshow("OpenCV Feed", frame)

    if cv2.waitKey(10) and 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

[ WARN:0@479.788] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@479.789] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


In [5]:
cap.release??

Object `cap.release` not found.


# 3. Extracting Keypoints

# 4. Setup Folders

# 5. Collect Keypoints for Training and Testing

# 6. Preprocessing Data

# 7. Build and Train LSTM model - Tensorflow

# 8. Make Predictions

# 9. Save Weights

# 10. Evaluation

# 11. Test in Real Time